## Prueba NG Solve

Simulación

Primero se debe llamar a la gui (graphical user interphace), así esta se abre y luego cualquier simulación se observará en ella

In [2]:

import netgen.gui    # Si se cierra la GUI, hay que hacerle Restart al notebook y volver a correr esta linea

In [3]:

# Importamos los paquetes
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *

def solve(h, draw = 0):
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))  # h será el tamaño de la malla y se usará el tipo unit_square, para otras geometrías hay que buscar el nombre y en base a ello se tendrán distintos boundries (se explicitan con strings ('bottom', 'top', etc))

    # Se definen funciones que se utilizarán, todas con lengauje simbólico  (acá están los ejemplos de la tarea).
    source = exp((x-0.5)*(y-0.5))*(2.5*x*y - y**4*x + y**3*x - 2*y**3*x - 2*y**3 - y**2*x**3 + y**2*x**2 - 0.5*y**2*x + y**2 - 3*y*x**2 - x)
    ud = exp((x-0.5)*(y-0.5))*x*y
    g = exp((x-0.5)*(y - 0.5))*(x*y + y**2*x - 0.5*y**2)


    fesp = H1(mesh, order=3, dirichlet="bottom")  # Se define un espacio de elementos finitos, H1 corresponde al espacio de polinomios continuos, se define también el tipo de condición según el borde (en este caso 'bottom')
    up, vp = fesp.TnT()                           # Se crean las funciones en el orden: trial function, test function

    #u = fes.TrialFunction()  # symbolic object   # Esta es una forma alternativa de generar la función de trial, test
    #v = fes.TestFunction()   # symbolic object
    #gfu = GridFunction(fes)  # solution

    ap = BilinearForm(fesp)                       # Se arma la forma bilineal (lado izquierdo de la formulación débil) (primero definiéndola para los elementos finitos diseñados)
    ap += (grad(up)*y*grad(vp) + up*vp)*dx        # Se coloca directamente la expresión
    ap.Assemble()                                 # Este paso es clave para armar la matriz que luego se usará en simular

    fp = LinearForm(fesp)                         # Se arma la forma lineal (lado derecho de la formulación débil)
    fp += source*vp*dx + g*vp * ds                # Se coloca directamente la expresión
    fp.Assemble()                                 # El ensamblaje del vector

    # con el comando 
    # ap.mat # se puede obtener la matriz
    # fp.vec # se puede obtener el vector

    gfup = GridFunction(fesp, "u-primal")         # Acá se guardará la solución
    gfup.Set(x + y, BND)                          # Parece que acá se agrega dirichlet

    gfup.Set(g, definedon=mesh.Boundaries("bottom"))                   # Y parece que acá se agregan directamente las condiciones de borde von neuomann

    r = fp.vec.CreateVector()                                            # Este vector permite que se puedan incorporar correctamente las condiciones de borde al sistema                
    r.data = fp.vec - ap.mat * gfup.vec                                  # Con esto    
    gfup.vec.data += ap.mat.Inverse(freedofs=fesp.FreeDofs()) * fp.vec   # Con esto se resuelve el sistema
    if draw:
        Draw (gfup)
    return gfup, mesh
#Draw (grad(gfup), mesh, "flux-primal")

In [4]:
gfup, mesh = solve(0.1, 1)

Análisis de convergencia